## 1. Introdução

### [Dataset](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package)

### Contexto
Predizer se vai ou não chover na Australia no próximo dia treinando um classificador binário

### Conteúdo
O dataset contém observações diárias do clima nas estações meteorológicas da Australia no período de 01/01/2009 até 25/06/2017, para maioria das estações.

O variável objetivo é a RainTomorrow que significa que choveu ou não no próximo dia.

### Kernels usados como referência
* [Análise do comportamento de cada variável e Cross Validation](https://www.kaggle.com/naumov1889/rain-in-australia-binary-classification)
* [Análise exploratória](https://www.kaggle.com/vanessaasoares/trabalho-ferramentas-ii)
* [Preparação dos dados](https://www.kaggle.com/aninditapani/will-it-rain-tomorrow)
* [Random Forest](https://www.kaggle.com/jmat94/basic-eda-and-randomforest-modelling/notebook)
* [Random Forest e Data Visualization](https://www.kaggle.com/ferneutron/classification-and-data-visualization)

---

### Considerações Iniciais
* Será usado o classificador [Random Forest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) - [For Humans](https://medium.com/machina-sapiens/o-algoritmo-da-floresta-aleat%C3%B3ria-3545f6babdf8)

## 2. Importação das bibliotecas

In [ ]:
#import pydotplus
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import numpy as np

from scipy import stats
from IPython.display import Image  
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.externals.six import StringIO  
from sklearn.tree import export_graphviz

## 3. Análise exploratória

### 3.2 Leitura do dataset

In [ ]:
df = pd.read_csv('../input/weatherAUS.csv')
df.info()

### 3.3 Análise da quantidade de registros para cada estação meteorológica

In [ ]:
df['Location'].value_counts()

### 3.4 Valores Nulos
Vamos nos livrar de colunas com quantidade significativa de valores nulos.

In [ ]:
porcentagem_dados_nulo = pd.Series(index=df.columns)

for coluna in df:
    porcentagem_dados_nulo[coluna] = df[coluna].count()/df.shape[0]
    
porcentagem_dados_nulo_ordenado = porcentagem_dados_nulo.sort_values()

print(porcentagem_dados_nulo_ordenado)

In [ ]:
porcentagem_dados_nulo_ordenado.plot.bar()

### 3.5 Distribuição dos dias que teve ou não chuva

In [ ]:
plot_sb = sbn.countplot(df.RainTomorrow, label='Total')
NotRain, Rain = df.RainTomorrow.value_counts()
print('Não Choveu: ',NotRain)
print('Choveu : ',Rain)

## 4. Preparação dos dados

### 4.1 Excluir colunas

As colunas Cloud9pm, Cloud3pm, Evaporation, e Sunshine serão excluidas por terem uma grande quantidade de valores nulos.

As colunas Location, Date serão excluídas pois não vamos levar em consideração a data e a localização. Queremos apenas saber se vai chover na Austrália.

Além disso é necessário excluir a coluna RISK_MM  como recomendação para não atrapalhar o modelo, conforme a [discussão](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package/discussion/78316)

In [ ]:
df = df.drop(columns=['Sunshine','Evaporation','Cloud3pm','Cloud9am','Location','RISK_MM','Date'],axis=1)

In [ ]:
df.shape

### 4.2 Exlcuir as linhas com valores nulos

In [ ]:
df = df.dropna(how='any')

In [ ]:
df.shape

### 4.3 Remoção de Outliers
[Sobre o Z Score](http://www.cpaqv.org/estatistica/teste_z.pdf)

![Z Score](http://www.ltcconline.net/greenl/courses/201/probdist/ctsdis8.jpg)

Nas estatísticas, um escore-z (ou escore padrão) de uma observação é o número de desvios padrão acima ou abaixo da média da população.

In [ ]:
z = np.abs(stats.zscore(df._get_numeric_data()))
print(z)
df = df[(z < 3).all(axis=1)]

In [ ]:
df.shape

### 4.4 Troca de valores
Para as colunas RainToday e RainTomorrow precisamos trocar os valores de Yes e No por 1 e 0 respectivamente

Depois precisamos transformar os valores não numéricos em valores numéricos. Para isso vamos usar a função [get_dummies](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html) do pandas

In [ ]:
df['RainToday'] = df['RainToday'].replace({'No': 0,'Yes': 1})
df['RainTomorrow'] = df['RainTomorrow'].replace({'No': 0,'Yes': 1})

In [ ]:
colunas_categoricas = ['WindGustDir', 'WindDir3pm', 'WindDir9am']
for col in colunas_categoricas:
    print(np.unique(df[col]))

# transformar as colunas de categoria
df = pd.get_dummies(df, columns=colunas_categoricas)

In [ ]:
df.head()

In [ ]:
df.shape

### 4.5 Padronização dos dados
Vamos colocar todos os dados em uma escala de 0 a 1

In [ ]:
scaler = preprocessing.MinMaxScaler()
scaler.fit(df)
df = pd.DataFrame(scaler.transform(df), index=df.index, columns=df.columns)

In [ ]:
df.head()

### 4.6 Separando as colunas
Nesta etapa vamos separar a coluna RainTomorrow do restante do dataset

Além disso vamos esolher as colunas mais relevantes usando a função SelectKBest com chi2, pois o uso das colunas mais relevantes diminui o efeito da geração de falsos negativos causados pela desbanleceamento.

In [ ]:
y = df.RainTomorrow
x = df.drop(['RainTomorrow'], axis=1)

In [ ]:
selector = SelectKBest(chi2, k=3)
selector.fit(x, y)
X_new = selector.transform(x)
print(x.columns[selector.get_support(indices=True)])

In [ ]:
x = df[['Humidity3pm','Rainfall','RainToday']]

### 4.7 Criar conjunto de teste e de treinamento
O conjunto de treinamento usa 75% do dataset.

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25)

## 5. Criação do Modelo

### 5.1 Criar o modelo com Random Forest
Foram usadas 50 árvores com profundidade de 3

In [ ]:
clf_rf = RandomForestClassifier(n_estimators=50, max_depth=3, random_state=23)
clf_rf.fit(x_train,y_train)

## 6. Análise dos Resultados

### 6.1 Precisão
O modelo apresentou uma precisão de 83%, sendo que quando os resultados são de Não Chuva, ele tem um desempenho melhor.

In [ ]:
y_pred = clf_rf.predict(x_test)
score = accuracy_score(y_test,y_pred)
print('Precisão: {:.3f}'.format(score))

In [ ]:
print(classification_report(y_test, y_pred))

### 6.2 Gerando a matriz de confusão
Pela matriz de confusão, podemos perceber que o modelo gerou um número considerável de falsos negativos.

In [ ]:
cm = confusion_matrix(y_test,y_pred)
fig, ax = plt.subplots(figsize=(12,12))
sbn.heatmap(cm,xticklabels=['No Rain','Rain'],yticklabels=['No Rain','Rain'],annot=True,fmt="d")

### 6.3 Cross Validation
Quando geramos o cross validation, os resultados das demais amostras são semelhantes.

In [ ]:
score_cv = cross_val_score(clf_rf, x, y, cv=20, scoring="f1")
print(score_cv)
print(score_cv.mean())
print(score_cv.std())

### 6.4 Árvore de Decisão Final

In [ ]:
treesDot = StringIO()
export_graphviz(clf_rf.estimators_[5], out_file=treesDot, filled=True, rounded=True, special_characters=True, class_names=["No Rain","Rain"], feature_names=x.columns.values)
#graph = pydotplus.graph_from_dot_data(treesDot.getvalue())  
#Image(graph.create_png())

![alt text](...)

## 7. Conclusão
O Modelo proposto com Random Forest teve um desempenho interessante de cerca de 83% na média.

O que chamou a atenção foi a quantidade de falsos negativos, ou seja, quando o modelo prediz que não vai chover e na realidade chove. Uma possível explicação é a distribuição desigual da amostra, pois existem muito mais exemplos de dias que não chovem do que dias que chovem.